# 3. INTERMEDIATE MACHINE LEARNING

# 3.2. MISSING VALUES

In [1]:
# Chargement des librairies
import pandas as pd

In [2]:
#  astuce pour afficher toutes les lignes sans les ...
pd.set_option('display.max_row', 80)
#  astuce pour afficher toutes les colonnes dans les head()
pd.set_option('display.max_column', 80)

## CHARGEMENT DES DATASETS FULL ET TEST

In [2]:
# Chargement des datasets
dataset_input_path = 'C:/Users/PC Maison/4-KAGGLE\KAGGLE_DEV/KAGGLE_COURS_3-MACHINE_LEARNING_INTERMEDIATE/'
X_full = pd.read_csv(dataset_input_path + 'home-data-for-ml-course/input/train.csv', index_col='Id')
X_test_full = pd.read_csv(dataset_input_path + 'home-data-for-ml-course/input/test.csv', index_col='Id')

In [6]:
# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

In [7]:
# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

## TRAIN SET et VAL SET

In [9]:
from sklearn.model_selection import train_test_split

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8,
                                                      test_size=0.2,
                                                      random_state=0)

In [10]:
# Voir les 5 premières lignes du train set
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,1774,1822,1828,0,0,1828,0,0,2,0,3,1,9,1,2007.0,3,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,894,894,894,0,0,894,0,0,1,0,2,1,5,0,1962.0,1,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,163,876,964,0,0,964,1,0,1,0,2,1,5,0,1921.0,2,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,350,1568,1689,0,0,1689,1,0,2,0,3,1,7,2,2002.0,3,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,1541,1541,1541,0,0,1541,0,0,2,0,3,1,6,1,2001.0,3,843,468,81,0,0,0,0,0,1,2006


In [11]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

(1168, 36)


In [60]:
# Number of missing values in each column of training data

nbr_lignes_tot=X_train.shape[0]
print('Nombre total de lignes : ', nbr_lignes_tot)

missing_val_count_by_column = (X_train.isnull().sum())
missing_val_name_column =  missing_val_count_by_column[missing_val_count_by_column > 0]
print('----------------------------------------------------------')
print('Valeurs manquantes : nom_colonne - nombre : \n' , missing_val_count_by_column[missing_val_count_by_column > 0])
sum_missing_values = missing_val_name_column.sum()
print('----------------------------------------------------------')
print('Nombre total de valeurs manquantes :', sum_missing_values)
print('----------------------------------------------------------')
print('%val manquante :', float('%0.2f' % ((sum_missing_values/nbr_lignes_tot)*100)), ' %')

# for i in range(len(missing_val_count_by_column)):
#     print('Colonne : ', missing_val_count_by_column[i], ' - % : ', missing_val_count_by_column[i]/nbr_lignes_tot)



Nombre total de lignes :  1168
----------------------------------------------------------
Valeurs manquantes : nom_colonne - nombre : 
 LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64
----------------------------------------------------------
Nombre total de valeurs manquantes : 276
----------------------------------------------------------
%val manquante : 23.63  %


##  SCORING

In [62]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## APPROCHE n°1 : DROP MISSING VALUES

In [63]:
# Fill in the line below: get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

## SCORING

In [64]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


## APPROCHE n°2 : IMPUTATION

In [65]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

## SCORING

In [66]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


## PREPROCESSING

In [68]:
# Imputation
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

# Imputation removed column names; put them back
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

## MODELISATION

In [69]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
17791.59899543379


## PREDICTION SUR LE TEST SET

In [70]:
# Fill in the line below: preprocess test data
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# Fill in the line below: get test predictions
preds_test = model.predict(final_X_test)